# Numerical Optmization Exam

In [1]:
import numpy as np

## Exercise 5.1

In [17]:
# Build the n×n Hilbert matrix  H_ij = 1/(i+j-1)
def hilbert(n):
    i = np.arange(1, n + 1)
    return 1.0 / (i[:, None] + i[None, :] - 1.0)

# Conjugate gradient method (algorithm 5.2)
def cg(A, b, x0, tol, maxit, verbose = False):
    """Solve  A x = b  with CG.

    Parameters
    ----------
    A : SPD matrix (n×n)
    b : right‑hand side (n,)
    x0 : starting point (defaults to zeros)
    tol : stop when  ‖r_k‖ ≤ tol ‖r_0‖
    maxit : maximum iterations 
    return_history : if True, also return list of residual norms

    Returns
    -------
    x      : approximate solution
    it     : iterations performed
    history: list of residual norms 
    """
   
    n = b.size
    x = x0.copy()

    r = A @ x - b           # r0 = A x0 – b 
    p = -r.copy()           # p0 = -r0
    rs_old = r @ r          # r'r

    res0 = np.sqrt(rs_old)
    history: list[float] = [res0]

    for k in range(1, maxit + 1):
        alpha = rs_old / (p @ (A @ p))      # (5.24a)
        x += alpha * p                      # (5.24b)
        r += alpha * (A @ p)                # (5.24c)

        rs_new = r @ r
        res = np.sqrt(rs_new)
        history.append(res)
        
        if verbose:
            print(f"k {k} -- res = {res}")

        if res <= tol * res0:
            return (x, k) 

        beta = rs_new / rs_old              # (5.24d)
        p = -r + beta * p                   # (5.24e)
        
        rs_old = rs_new

    return (x, maxit)


# Exercise 5.1 driver
def run():
    dims = [5, 8, 12, 20]
    tol = 1e-6
    print("n   iterations ")
    print("-   ---------- ")

    for n in dims:
        H = hilbert(n)
        b = np.ones(n)
        x0 = np.zeros(n)

        _, it = cg(H, b, x0, tol=tol, maxit=100)
        print(f"{n:<3d} {it:^10d}")

In [18]:
run()

n   iterations 
-   ---------- 
5       6     
8       19    
12      37    
20      68    


## Exercise 14.15

In [17]:
"""Mehrotra predictor–corrector interior‑point LP solver
    *with every Newton step solved as the FULL KKT system* (Eq. 14.30).

This file keeps everything in **one function** (`mehrotra_lp`) exactly as you
asked – no separate helpers, no normal‑equation reduction.  In each Newton
iteration we build the dense

        ┌            ┐
        │ 0  Aᵀ  I  │
    K = │ A   0   0 │   (14.30)
        │ S   0   X │
        └            ┘

and solve it with `numpy.linalg.solve` both for the *affine‑scaling* (predictor)
step and the *combined predictor+corrector* step.

At the bottom of the file you'll find a **tiny demo** that builds a random LP
following the textbook’s recipe (your screenshot) and calls the solver.
"""

# ---------------------------------------------------------------------------
# Random LP generator (recipe from the book)
# ---------------------------------------------------------------------------

def generate_random_lp(m: int, n: int, *, seed: int | None = None):
    """Return (A, b, c, x⋆) where x⋆ is the known optimum.

    The construction is exactly what your screenshot specifies:

        x⋆_i > 0  for i = 1..m,   x⋆_i = 0  otherwise
        s⋆_i > 0  for i = m+1..n,  s⋆_i = 0  otherwise
        λ⋆  random,  c = Aᵀ λ⋆ + s⋆,  b = A x⋆.
    """
    rng = np.random.default_rng(seed)

    A = rng.standard_normal((m, n))

    x_star = np.zeros(n)
    x_star[:m] = rng.random(m) + 0.5  # positive

    s_star = np.zeros(n)
    s_star[m:] = rng.random(n - m) + 0.5  # positive

    lam_star = rng.standard_normal(m)

    c = A.T @ lam_star + s_star
    b = A @ x_star

    return A, b, c, x_star


# ---------------------------------------------------------------------------
# Mehrotra predictor–corrector IPM (ALL full KKT solves, one function)
# ---------------------------------------------------------------------------

def mehrotra_lp(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    *,
    max_iter: int = 50,
    tol: float = 1e-8,
    eta: float = 0.99,
    verbose: bool = False,
):
    """Solve  min cᵀx  s.t.  Ax = b,  x ≥ 0  using Algorithm 14.3.

    Every Newton system – both predictor and corrector – is solved as the FULL
    KKT system of Eq. (14.30).  Returns (x, λ, s).
    """
    m, n = A.shape

    # Strictly positive starting point – simple but effective.
    x   = np.full(n, 10.0)
    s   = np.full(n, 10.0)
    lam = np.zeros(m)

    e = np.ones(n)

    for k in range(max_iter):
        r_b = A @ x - b
        r_c = A.T @ lam + s - c
        mu   = (x @ s) / n

        if verbose:
            print(f"iter {k:2d}  ‖r_b‖={np.linalg.norm(r_b):.1e}  ‖r_c‖={np.linalg.norm(r_c):.1e}  μ={mu:.1e}")

        if max(np.linalg.norm(r_b), np.linalg.norm(r_c), mu) < tol:
            break

        # ------------------------------------------------------------------
        # 1) Predictor (affine‑scaling) step – FULL KKT solve
        # ------------------------------------------------------------------
        X = np.diag(x)
        S = np.diag(s)
        K = np.block([
            [np.zeros((n, n)),   A.T,               np.eye(n)],
            [A,                  np.zeros((m, m)),  np.zeros((m, n))],
            [S,                  np.zeros((n, m)),  X],
        ])
        rhs_aff = np.concatenate([-r_c, -r_b, -x * s])
        delta_aff = np.linalg.solve(K, rhs_aff)
        dx_aff   = delta_aff[:n]
        dlam_aff = delta_aff[n:n + m]
        ds_aff   = delta_aff[n + m:]

        # Step lengths (14.32) ---------------------------------------------
        alpha_aff_pri = 1.0
        neg_dx = dx_aff < 0
        if np.any(neg_dx):
            alpha_aff_pri = min(1.0, np.min(-x[neg_dx] / dx_aff[neg_dx]))

        alpha_aff_dua = 1.0
        neg_ds = ds_aff < 0
        if np.any(neg_ds):
            alpha_aff_dua = min(1.0, np.min(-s[neg_ds] / ds_aff[neg_ds]))

        mu_aff = ((x + alpha_aff_pri * dx_aff) @ (s + alpha_aff_dua * ds_aff)) / n
        sigma  = (mu_aff / mu) ** 3  # (14.34)

        # ------------------------------------------------------------------
        # 2) Corrector + centering step – FULL KKT with modified RHS
        # ------------------------------------------------------------------
        rxs = dx_aff * ds_aff  # element‑wise
        rhs_cor = np.concatenate([
            -r_c,
            -r_b,
            -x * s - rxs + sigma * mu * e,
        ])
        delta = np.linalg.solve(K, rhs_cor)
        dx   = delta[:n]
        dlam = delta[n:n + m]
        ds   = delta[n + m:]

        # Fraction‑to‑boundary rule (14.36–14.38) --------------------------
        alpha_pri_max = 1.0
        neg_dx = dx < 0
        if np.any(neg_dx):
            alpha_pri_max = min(alpha_pri_max, np.min(-x[neg_dx] / dx[neg_dx]))

        alpha_dua_max = 1.0
        neg_ds = ds < 0
        if np.any(neg_ds):
            alpha_dua_max = min(alpha_dua_max, np.min(-s[neg_ds] / ds[neg_ds]))

        alpha_pri = eta * alpha_pri_max
        alpha_dua = eta * alpha_dua_max

        # Update ------------------------------------------------------------
        x   += alpha_pri * dx
        lam += alpha_dua * dlam
        s   += alpha_dua * ds

    return x, lam, s

In [19]:
m, n = 6, 6  
A, b, c, x_true = generate_random_lp(m, n, seed=42)
x_opt, lam_opt, s_opt = mehrotra_lp(A, b, c, verbose=True)
print("\nOptimal objective value:", c @ x_opt)
print("Distance to true x     :", np.linalg.norm(x_opt - x_true))

iter  0  ‖r_b‖=3.2e+01  ‖r_c‖=2.3e+01  μ=1.0e+02
iter  1  ‖r_b‖=3.2e-01  ‖r_c‖=2.3e-01  μ=8.5e+00
iter  2  ‖r_b‖=3.2e-03  ‖r_c‖=2.3e-03  μ=1.4e-01
iter  3  ‖r_b‖=3.2e-05  ‖r_c‖=2.3e-05  μ=1.4e-03
iter  4  ‖r_b‖=3.2e-07  ‖r_c‖=2.3e-07  μ=1.4e-05
iter  5  ‖r_b‖=3.2e-09  ‖r_c‖=2.3e-09  μ=1.4e-07
iter  6  ‖r_b‖=3.2e-11  ‖r_c‖=2.3e-11  μ=1.4e-09

Optimal objective value: 4.480502807601511
Distance to true x     : 2.2146301837596377e-11
